# State Pattern Tutorial 🎰

## What is the State Pattern?

The State Pattern allows an object to alter its behavior when its internal state changes. The object will appear to change its class. It encapsulates state-based behavior and delegates behavior to the current state object.

**Real-world analogy**: Think of a vending machine. It behaves differently depending on its current state: when it has no money inserted, it displays "Insert Coins"; when money is inserted, it displays "Select Product"; when a product is selected, it either dispenses the product or asks for more money. The same button press does different things based on the machine's state.

## Table of Contents
1. [What is the State Pattern?](#what-is-the-state-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [Document Workflow System (Advanced)](#advanced-example-document-workflow-system)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## Why Do We Need It?

Let's see what happens when we manage state with conditional statements:

In [ ]:
# Problem: Managing state with conditional statements

class BadVendingMachine:
    def __init__(self):
        self.state = "NO_COIN"  # States: NO_COIN, HAS_COIN, PRODUCT_SELECTED, OUT_OF_STOCK
        self.coins_inserted = 0
        self.product_price = 150  # 1.50 in cents
        self.inventory = 5
        self.selected_product = None
    
    def insert_coin(self, amount):
        if self.state == "NO_COIN":
            self.coins_inserted += amount
            self.state = "HAS_COIN"
            print(f"Coin inserted. Total: {self.coins_inserted} cents")
        elif self.state == "HAS_COIN":
            self.coins_inserted += amount
            print(f"More coins inserted. Total: {self.coins_inserted} cents")
        elif self.state == "PRODUCT_SELECTED":
            self.coins_inserted += amount
            print(f"More coins inserted. Total: {self.coins_inserted} cents")
            if self.coins_inserted >= self.product_price:
                self._dispense_product()
        elif self.state == "OUT_OF_STOCK":
            print("Machine is out of stock. Coin returned.")
            # Return coin logic
        else:
            print("Invalid state for coin insertion")
    
    def select_product(self, product):
        if self.state == "NO_COIN":
            print("Please insert coins first")
        elif self.state == "HAS_COIN":
            if self.inventory <= 0:
                self.state = "OUT_OF_STOCK"
                print("Product out of stock")
                return
            
            self.selected_product = product
            self.state = "PRODUCT_SELECTED"
            print(f"Product selected: {product}")
            
            if self.coins_inserted >= self.product_price:
                self._dispense_product()
            else:
                needed = self.product_price - self.coins_inserted
                print(f"Please insert {needed} more cents")
        elif self.state == "PRODUCT_SELECTED":
            print("Product already selected. Please insert more coins or cancel.")
        elif self.state == "OUT_OF_STOCK":
            print("Machine is out of stock")
    
    def cancel(self):
        if self.state == "NO_COIN":
            print("Nothing to cancel")
        elif self.state == "HAS_COIN" or self.state == "PRODUCT_SELECTED":
            print(f"Transaction cancelled. Returning {self.coins_inserted} cents")
            self.coins_inserted = 0
            self.selected_product = None
            self.state = "NO_COIN"
        elif self.state == "OUT_OF_STOCK":
            print("Machine out of stock. No transaction to cancel.")
    
    def _dispense_product(self):
        if self.coins_inserted >= self.product_price and self.inventory > 0:
            change = self.coins_inserted - self.product_price
            print(f"Dispensing {self.selected_product}")
            if change > 0:
                print(f"Returning change: {change} cents")
            
            self.inventory -= 1
            self.coins_inserted = 0
            self.selected_product = None
            
            if self.inventory <= 0:
                self.state = "OUT_OF_STOCK"
            else:
                self.state = "NO_COIN"

# Problems with this approach:
# 1. Complex nested if-statements
# 2. State logic scattered across methods
# 3. Hard to add new states or modify behavior
# 4. Violates Single Responsibility Principle
# 5. Difficult to test individual state behaviors

# Let's see it in action
print("--- Bad Vending Machine Example ---")
machine = BadVendingMachine()
machine.select_product("Soda")  # Should ask for coins
machine.insert_coin(50)
machine.select_product("Soda")
machine.insert_coin(100)  # Should dispense

## Simple Implementation

Let's solve this with the State Pattern:

In [ ]:
from abc import ABC, abstractmethod

# Context class
class VendingMachine:
    def __init__(self):
        self.coins_inserted = 0
        self.product_price = 150  # 1.50 in cents
        self.inventory = 5
        self.selected_product = None
        
        # State objects
        self.no_coin_state = NoCoinState(self)
        self.has_coin_state = HasCoinState(self)
        self.product_selected_state = ProductSelectedState(self)
        self.out_of_stock_state = OutOfStockState(self)
        
        # Set initial state
        self.current_state = self.no_coin_state
    
    def set_state(self, state):
        self.current_state = state
        print(f"State changed to: {state.__class__.__name__}")
    
    def insert_coin(self, amount):
        self.current_state.insert_coin(amount)
    
    def select_product(self, product):
        self.current_state.select_product(product)
    
    def cancel(self):
        self.current_state.cancel()
    
    def dispense(self):
        self.current_state.dispense()
    
    def refill(self, amount):
        self.inventory = amount
        print(f"Machine refilled with {amount} products")
        if self.current_state == self.out_of_stock_state:
            self.set_state(self.no_coin_state)

# State interface
class VendingMachineState(ABC):
    def __init__(self, machine: VendingMachine):
        self.machine = machine
    
    @abstractmethod
    def insert_coin(self, amount):
        pass
    
    @abstractmethod
    def select_product(self, product):
        pass
    
    @abstractmethod
    def cancel(self):
        pass
    
    @abstractmethod
    def dispense(self):
        pass

# Concrete states
class NoCoinState(VendingMachineState):
    def insert_coin(self, amount):
        self.machine.coins_inserted += amount
        print(f"Coin inserted. Total: {self.machine.coins_inserted} cents")
        self.machine.set_state(self.machine.has_coin_state)
    
    def select_product(self, product):
        print("Please insert coins first")
    
    def cancel(self):
        print("Nothing to cancel")
    
    def dispense(self):
        print("Please insert coins and select a product first")

class HasCoinState(VendingMachineState):
    def insert_coin(self, amount):
        self.machine.coins_inserted += amount
        print(f"More coins inserted. Total: {self.machine.coins_inserted} cents")
    
    def select_product(self, product):
        if self.machine.inventory <= 0:
            print("Product out of stock")
            self.machine.set_state(self.machine.out_of_stock_state)
            return
        
        self.machine.selected_product = product
        print(f"Product selected: {product}")
        self.machine.set_state(self.machine.product_selected_state)
        
        # Check if we have enough money
        if self.machine.coins_inserted >= self.machine.product_price:
            self.machine.dispense()
        else:
            needed = self.machine.product_price - self.machine.coins_inserted
            print(f"Please insert {needed} more cents")
    
    def cancel(self):
        print(f"Transaction cancelled. Returning {self.machine.coins_inserted} cents")
        self.machine.coins_inserted = 0
        self.machine.set_state(self.machine.no_coin_state)
    
    def dispense(self):
        print("Please select a product first")

class ProductSelectedState(VendingMachineState):
    def insert_coin(self, amount):
        self.machine.coins_inserted += amount
        print(f"More coins inserted. Total: {self.machine.coins_inserted} cents")
        
        if self.machine.coins_inserted >= self.machine.product_price:
            self.machine.dispense()
    
    def select_product(self, product):
        print("Product already selected. Please insert more coins or cancel.")
    
    def cancel(self):
        print(f"Transaction cancelled. Returning {self.machine.coins_inserted} cents")
        self.machine.coins_inserted = 0
        self.machine.selected_product = None
        self.machine.set_state(self.machine.no_coin_state)
    
    def dispense(self):
        if self.machine.coins_inserted >= self.machine.product_price:
            change = self.machine.coins_inserted - self.machine.product_price
            print(f"Dispensing {self.machine.selected_product}")
            
            if change > 0:
                print(f"Returning change: {change} cents")
            
            self.machine.inventory -= 1
            self.machine.coins_inserted = 0
            self.machine.selected_product = None
            
            if self.machine.inventory <= 0:
                self.machine.set_state(self.machine.out_of_stock_state)
            else:
                self.machine.set_state(self.machine.no_coin_state)
        else:
            needed = self.machine.product_price - self.machine.coins_inserted
            print(f"Insufficient funds. Please insert {needed} more cents")

class OutOfStockState(VendingMachineState):
    def insert_coin(self, amount):
        print("Machine is out of stock. Coin returned.")
        # In a real machine, this would physically return the coin
    
    def select_product(self, product):
        print("Machine is out of stock")
    
    def cancel(self):
        print("Machine out of stock. No transaction to cancel.")
    
    def dispense(self):
        print("Cannot dispense. Machine is out of stock.")

# Testing the State Pattern implementation
print("--- Vending Machine with State Pattern ---")

machine = VendingMachine()

# Try to select without coins
machine.select_product("Soda")

# Insert coin and select
machine.insert_coin(50)
machine.select_product("Soda")

# Insert more coins to complete purchase
machine.insert_coin(100)

print("\n--- New transaction ---")
# Start new transaction
machine.insert_coin(200)  # More than enough
machine.select_product("Chips")

print("\n--- Cancel transaction ---")
# Cancel transaction
machine.insert_coin(75)
machine.cancel()

## Understanding the Implementation

### Key Concepts:

1. **Context**: The class whose behavior changes based on state (VendingMachine)
2. **State Interface**: Defines methods that all concrete states must implement
3. **Concrete States**: Implement state-specific behavior
4. **State Transitions**: States can change the context's current state

### Benefits:
- **Single Responsibility**: Each state handles its own behavior
- **Open/Closed**: Easy to add new states without modifying existing code
- **Eliminates Conditionals**: No more complex if-else chains
- **Testability**: Each state can be tested independently

## Advanced Example: Document Workflow System

Let's create a more complex example with a document workflow system:

In [ ]:
from datetime import datetime
from typing import Optional, List

# Context - Document with workflow states
class Document:
    def __init__(self, title: str, content: str, author: str):
        self.title = title
        self.content = content
        self.author = author
        self.created_at = datetime.now()
        self.current_reviewer = None
        self.comments: List[str] = []
        self.version = 1
        
        # State objects
        self.draft_state = DraftState(self)
        self.review_state = ReviewState(self)
        self.approved_state = ApprovedState(self)
        self.published_state = PublishedState(self)
        self.rejected_state = RejectedState(self)
        self.archived_state = ArchivedState(self)
        
        # Start in draft state
        self.current_state = self.draft_state
        print(f"Document '{self.title}' created in Draft state")
    
    def set_state(self, state):
        old_state = self.current_state.__class__.__name__
        self.current_state = state
        new_state = self.current_state.__class__.__name__
        print(f"Document state changed: {old_state} → {new_state}")
    
    def submit_for_review(self, reviewer: str):
        self.current_state.submit_for_review(reviewer)
    
    def approve(self, approver: str):
        self.current_state.approve(approver)
    
    def reject(self, reviewer: str, reason: str):
        self.current_state.reject(reviewer, reason)
    
    def publish(self):
        self.current_state.publish()
    
    def edit(self, new_content: str):
        self.current_state.edit(new_content)
    
    def archive(self):
        self.current_state.archive()
    
    def add_comment(self, comment: str):
        self.comments.append(f"[{datetime.now().strftime('%Y-%m-%d %H:%M')}] {comment}")
    
    def get_status(self) -> str:
        return self.current_state.__class__.__name__.replace('State', '')
    
    def __str__(self):
        return f"Document: {self.title} (v{self.version}) - Status: {self.get_status()}"

# State interface
class DocumentState(ABC):
    def __init__(self, document: Document):
        self.document = document
    
    @abstractmethod
    def submit_for_review(self, reviewer: str):
        pass
    
    @abstractmethod
    def approve(self, approver: str):
        pass
    
    @abstractmethod
    def reject(self, reviewer: str, reason: str):
        pass
    
    @abstractmethod
    def publish(self):
        pass
    
    @abstractmethod
    def edit(self, new_content: str):
        pass
    
    @abstractmethod
    def archive(self):
        pass

# Concrete states
class DraftState(DocumentState):
    def submit_for_review(self, reviewer: str):
        self.document.current_reviewer = reviewer
        self.document.add_comment(f"Submitted for review by {reviewer}")
        self.document.set_state(self.document.review_state)
    
    def approve(self, approver: str):
        print("Cannot approve a document in draft state. Submit for review first.")
    
    def reject(self, reviewer: str, reason: str):
        print("Cannot reject a document in draft state.")
    
    def publish(self):
        print("Cannot publish a draft document. Submit for review and approval first.")
    
    def edit(self, new_content: str):
        old_content = self.document.content
        self.document.content = new_content
        self.document.version += 1
        self.document.add_comment(f"Document edited by {self.document.author}")
        print(f"Document content updated (v{self.document.version})")
    
    def archive(self):
        self.document.add_comment("Document archived from draft state")
        self.document.set_state(self.document.archived_state)

class ReviewState(DocumentState):
    def submit_for_review(self, reviewer: str):
        print("Document is already under review.")
    
    def approve(self, approver: str):
        self.document.add_comment(f"Approved by {approver}")
        self.document.set_state(self.document.approved_state)
    
    def reject(self, reviewer: str, reason: str):
        self.document.add_comment(f"Rejected by {reviewer}: {reason}")
        self.document.current_reviewer = None
        self.document.set_state(self.document.rejected_state)
    
    def publish(self):
        print("Cannot publish document under review. Wait for approval.")
    
    def edit(self, new_content: str):
        print("Cannot edit document under review. Document moved back to draft.")
        self.document.current_reviewer = None
        self.document.set_state(self.document.draft_state)
        self.document.edit(new_content)  # Now edit in draft state
    
    def archive(self):
        self.document.add_comment("Document archived from review state")
        self.document.current_reviewer = None
        self.document.set_state(self.document.archived_state)

class ApprovedState(DocumentState):
    def submit_for_review(self, reviewer: str):
        print("Document is already approved.")
    
    def approve(self, approver: str):
        print("Document is already approved.")
    
    def reject(self, reviewer: str, reason: str):
        print("Cannot reject an approved document.")
    
    def publish(self):
        self.document.add_comment("Document published")
        self.document.set_state(self.document.published_state)
    
    def edit(self, new_content: str):
        print("Editing approved document moves it back to draft state.")
        self.document.set_state(self.document.draft_state)
        self.document.edit(new_content)
    
    def archive(self):
        self.document.add_comment("Approved document archived")
        self.document.set_state(self.document.archived_state)

class PublishedState(DocumentState):
    def submit_for_review(self, reviewer: str):
        print("Cannot submit published document for review.")
    
    def approve(self, approver: str):
        print("Document is already published.")
    
    def reject(self, reviewer: str, reason: str):
        print("Cannot reject a published document.")
    
    def publish(self):
        print("Document is already published.")
    
    def edit(self, new_content: str):
        print("Cannot edit published document directly. Create a new version.")
    
    def archive(self):
        self.document.add_comment("Published document archived")
        self.document.set_state(self.document.archived_state)

class RejectedState(DocumentState):
    def submit_for_review(self, reviewer: str):
        print("Moving rejected document back to review.")
        self.document.current_reviewer = reviewer
        self.document.add_comment(f"Resubmitted for review by {reviewer}")
        self.document.set_state(self.document.review_state)
    
    def approve(self, approver: str):
        print("Cannot approve rejected document. Submit for review first.")
    
    def reject(self, reviewer: str, reason: str):
        print("Document is already rejected.")
    
    def publish(self):
        print("Cannot publish rejected document.")
    
    def edit(self, new_content: str):
        print("Editing rejected document moves it to draft state.")
        self.document.set_state(self.document.draft_state)
        self.document.edit(new_content)
    
    def archive(self):
        self.document.add_comment("Rejected document archived")
        self.document.set_state(self.document.archived_state)

class ArchivedState(DocumentState):
    def submit_for_review(self, reviewer: str):
        print("Cannot submit archived document for review.")
    
    def approve(self, approver: str):
        print("Cannot approve archived document.")
    
    def reject(self, reviewer: str, reason: str):
        print("Cannot reject archived document.")
    
    def publish(self):
        print("Cannot publish archived document.")
    
    def edit(self, new_content: str):
        print("Cannot edit archived document.")
    
    def archive(self):
        print("Document is already archived.")

# Testing the document workflow system
print("--- Document Workflow System ---")

# Create a new document
doc = Document("Design Patterns Guide", "Introduction to design patterns...", "Alice")
print(doc)

print("\n--- Editing in draft ---")
doc.edit("Updated introduction to design patterns with examples...")

print("\n--- Submit for review ---")
doc.submit_for_review("Bob")

print("\n--- Try to edit during review ---")
doc.edit("Another update...")

print("\n--- Back to review and approve ---")
doc.submit_for_review("Bob")
doc.approve("Bob")

print("\n--- Publish ---")
doc.publish()
print(doc)

print("\n--- Try to edit published ---")
doc.edit("Can't edit this!")

print("\n--- Archive ---")
doc.archive()
print(doc)

print("\n--- Document comments ---")
for comment in doc.comments:
    print(f"  {comment}")

## State Pattern with State Machines

Let's create a more formal state machine example:

In [ ]:
from enum import Enum
from typing import Dict, Set, Callable

class OrderStatus(Enum):
    PENDING = "pending"
    CONFIRMED = "confirmed"
    PROCESSING = "processing"
    SHIPPED = "shipped"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"
    RETURNED = "returned"

class OrderStateMachine:
    def __init__(self):
        # Define valid state transitions
        self.transitions: Dict[OrderStatus, Set[OrderStatus]] = {
            OrderStatus.PENDING: {OrderStatus.CONFIRMED, OrderStatus.CANCELLED},
            OrderStatus.CONFIRMED: {OrderStatus.PROCESSING, OrderStatus.CANCELLED},
            OrderStatus.PROCESSING: {OrderStatus.SHIPPED, OrderStatus.CANCELLED},
            OrderStatus.SHIPPED: {OrderStatus.DELIVERED, OrderStatus.RETURNED},
            OrderStatus.DELIVERED: {OrderStatus.RETURNED},
            OrderStatus.CANCELLED: set(),  # Terminal state
            OrderStatus.RETURNED: set()    # Terminal state
        }
        
        # Define entry actions for each state
        self.entry_actions: Dict[OrderStatus, Callable] = {
            OrderStatus.PENDING: lambda: print("📝 Order created and waiting for confirmation"),
            OrderStatus.CONFIRMED: lambda: print("✅ Order confirmed, payment processed"),
            OrderStatus.PROCESSING: lambda: print("🏭 Order is being processed"),
            OrderStatus.SHIPPED: lambda: print("📦 Order shipped"),
            OrderStatus.DELIVERED: lambda: print("🏠 Order delivered"),
            OrderStatus.CANCELLED: lambda: print("❌ Order cancelled"),
            OrderStatus.RETURNED: lambda: print("↩️ Order returned")
        }
    
    def can_transition(self, from_state: OrderStatus, to_state: OrderStatus) -> bool:
        return to_state in self.transitions.get(from_state, set())
    
    def get_valid_transitions(self, from_state: OrderStatus) -> Set[OrderStatus]:
        return self.transitions.get(from_state, set())
    
    def execute_entry_action(self, state: OrderStatus):
        action = self.entry_actions.get(state)
        if action:
            action()

class Order:
    def __init__(self, order_id: str, items: List[str]):
        self.order_id = order_id
        self.items = items
        self.current_status = OrderStatus.PENDING
        self.state_machine = OrderStateMachine()
        self.history = []
        
        # Execute entry action for initial state
        self.state_machine.execute_entry_action(self.current_status)
        self._add_to_history("Order created")
    
    def transition_to(self, new_status: OrderStatus, reason: str = ""):
        if self.state_machine.can_transition(self.current_status, new_status):
            old_status = self.current_status
            self.current_status = new_status
            
            print(f"Order {self.order_id}: {old_status.value} → {new_status.value}")
            self.state_machine.execute_entry_action(new_status)
            
            history_entry = f"Status changed to {new_status.value}"
            if reason:
                history_entry += f" ({reason})"
            self._add_to_history(history_entry)
            
            return True
        else:
            print(f"❌ Invalid transition: {self.current_status.value} → {new_status.value}")
            return False
    
    def confirm(self, payment_method: str = ""):
        reason = f"Payment via {payment_method}" if payment_method else ""
        return self.transition_to(OrderStatus.CONFIRMED, reason)
    
    def start_processing(self):
        return self.transition_to(OrderStatus.PROCESSING)
    
    def ship(self, tracking_number: str = ""):
        reason = f"Tracking: {tracking_number}" if tracking_number else ""
        return self.transition_to(OrderStatus.SHIPPED, reason)
    
    def deliver(self):
        return self.transition_to(OrderStatus.DELIVERED)
    
    def cancel(self, reason: str = ""):
        return self.transition_to(OrderStatus.CANCELLED, reason)
    
    def return_order(self, reason: str = ""):
        return self.transition_to(OrderStatus.RETURNED, reason)
    
    def get_available_actions(self) -> Set[OrderStatus]:
        return self.state_machine.get_valid_transitions(self.current_status)
    
    def _add_to_history(self, event: str):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.history.append(f"[{timestamp}] {event}")
    
    def print_status(self):
        print(f"\n📋 Order {self.order_id}")
        print(f"   Items: {', '.join(self.items)}")
        print(f"   Status: {self.current_status.value}")
        
        available = self.get_available_actions()
        if available:
            actions = [status.value for status in available]
            print(f"   Available actions: {', '.join(actions)}")
        else:
            print(f"   No further actions available (terminal state)")
    
    def print_history(self):
        print(f"\n📖 Order {self.order_id} History:")
        for event in self.history:
            print(f"   {event}")

# Testing the order state machine
print("--- Order State Machine Example ---")

# Create a new order
order = Order("ORD-001", ["Laptop", "Mouse", "Keyboard"])
order.print_status()

print("\n--- Normal flow ---")
order.confirm("Credit Card")
order.start_processing()
order.ship("TRACK123456")
order.deliver()

order.print_status()
order.print_history()

print("\n--- New order with cancellation ---")
order2 = Order("ORD-002", ["Phone", "Case"])
order2.confirm("PayPal")
order2.cancel("Customer requested cancellation")

order2.print_status()

print("\n--- Try invalid transition ---")
order2.ship()  # Should fail - can't ship a cancelled order

print("\n--- Order with return ---")
order3 = Order("ORD-003", ["Book"])
order3.confirm("Debit Card")
order3.start_processing()
order3.ship("TRACK789")
order3.deliver()
order3.return_order("Product defective")

order3.print_history()

## When to Use (and When NOT to Use)

### Use State Pattern when:
- An object's behavior depends on its state and must change at runtime
- You have complex conditional statements that check the object's state
- You need to avoid duplicate code in similar states
- The number of states is finite and well-defined
- State transitions follow clear rules
- You want to make states explicit in your code

### Don't use when:
- You have only a few states with simple behavior
- State changes are rare or the logic is simple
- The overhead of creating state classes isn't justified
- States don't have significantly different behaviors

### Real-world applications:
- Workflow systems (document approval, order processing)
- Game development (player states, AI behaviors)
- UI components (button states: normal, hover, pressed, disabled)
- Network protocols (TCP connection states)
- Media players (playing, paused, stopped, buffering)
- Finite state machines in embedded systems
- Approval workflows in business applications
- Shopping cart and checkout processes
- User authentication states (logged out, logged in, expired)
- Device control systems (washing machines, elevators)

### State Pattern vs Strategy Pattern:
- **State**: Object behavior changes based on internal state; states often know about each other
- **Strategy**: Algorithm selection is external; strategies are independent of each other